In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch


In [2]:
vietnamnet = {
    "Đời sống":
    ["https://vietnamnet.vn/doi-song/gia-dinh",
     "https://vietnamnet.vn/doi-song/chuyen-la",
     "https://vietnamnet.vn/doi-song/gioi-tre",
     "https://vietnamnet.vn/doi-song/tam-su",
     "https://vietnamnet.vn/doi-song/am-thuc",
     "https://vietnamnet.vn/doi-song/meo-vat", ],
    "Thời sự":
    ["https://vietnamnet.vn/thoi-su/chong-tham-nhung",
     "https://vietnamnet.vn/thoi-su/quoc-hoi-khoa-xv",
     "https://vietnamnet.vn/thoi-su/an-toan-giao-thong",
     "https://vietnamnet.vn/thoi-su/moi-truong",
     "https://vietnamnet.vn/thoi-su/clip-nong",
     "https://vietnamnet.vn/thoi-su/bhxh-bhyt",
     "https://vietnamnet.vn/thoi-su/quoc-phong", ],
    "Quốc tế":
    ["https://vietnamnet.vn/the-gioi/binh-luan-quoc-te",
     "https://vietnamnet.vn/the-gioi/chan-dung",
     "https://vietnamnet.vn/the-gioi/ho-so",
     "https://vietnamnet.vn/the-gioi/the-gioi-do-day",
     "https://vietnamnet.vn/the-gioi/viet-nam-va-the-gioi",
     "https://vietnamnet.vn/the-gioi/quan-su", ],
    "Kinh doanh":
    ["https://vietnamnet.vn/kinh-doanh",
     "https://vietnamnet.vn/kinh-doanh/tai-chinh",
     "https://vietnamnet.vn/kinh-doanh/dau-tu",
     "https://vietnamnet.vn/kinh-doanh/thi-truong",
     "https://vietnamnet.vn/kinh-doanh/doanh-nhan",
     "https://vietnamnet.vn/kinh-doanh/tu-van-tai-chinh",
     "https://vietnamnet.vn/kinh-doanh/diem-tin-dung-cong-dan-4-0", ],
    "Giải trí":
    ["https://vietnamnet.vn/giai-tri",
     "https://vietnamnet.vn/giai-tri/the-gioi-sao",
     "https://vietnamnet.vn/giai-tri/thoi-trang",
     "https://vietnamnet.vn/giai-tri/nhac",
     "https://vietnamnet.vn/giai-tri/phim",
     "https://vietnamnet.vn/giai-tri/truyen-hinh",
     "https://vietnamnet.vn/giai-tri/sach",
     "https://vietnamnet.vn/giai-tri/di-san-my-thuat-san-khau", ],
    "Thể thao":
    ["https://vietnamnet.vn/thong-tin-unesco",
     "https://vietnamnet.vn/the-thao",
     "https://vietnamnet.vn/the-thao/bong-da-viet-nam",
     "https://vietnamnet.vn/the-thao/bong-da-quoc-te",
     "https://vietnamnet.vn/the-thao/hau-truong",
     "https://vietnamnet.vn/the-thao/cac-mon-khac",
     "https://vietnamnet.vn/the-thao/xem-truc-tiep-bong-da",
     "https://vietnamnet.vn/the-thao/du-lieu-bong-da",
     "https://vietnamnet.vn/the-thao/tin-chuyen-nhuong",
     "https://vietnamnet.vn/the-thao/video-the-thao", ],
    "Sức khỏe":
    ["https://vietnamnet.vn/suc-khoe",
     "https://vietnamnet.vn/suc-khoe/lam-dep",
     "https://vietnamnet.vn/suc-khoe/tu-van-suc-khoe",
     "https://vietnamnet.vn/suc-khoe/benh", ],
    "Pháp luật":
    ["https://vietnamnet.vn/phap-luat",
     "https://vietnamnet.vn/phap-luat/ho-so-vu-an",
     "https://vietnamnet.vn/phap-luat/tu-van-phap-luat",
     "https://vietnamnet.vn/phap-luat/ky-su-phap-dinh", ],
    "Oto - xe máy":
    ["https://vietnamnet.vn/oto-xe-may",
     "https://vietnamnet.vn/oto-xe-may/xe-moi",
     "https://vietnamnet.vn/oto-xe-may/kham-pha",
     "https://vietnamnet.vn/oto-xe-may/sau-tay-lai",
     "https://vietnamnet.vn/oto-xe-may/dien-dan",
     "https://vietnamnet.vn/oto-xe-may/tu-van",
     "https://vietnamnet.vn/oto-xe-may/danh-gia-xe",
     "https://vietnamnet.vn/oto-xe-may/gia-xe", ],
    "Công nghệ":
    ["https://vietnamnet.vn/cong-nghe",
     "https://vietnamnet.vn/cong-nghe/cong-dong-mang",
     "https://vietnamnet.vn/cong-nghe/ung-dung",
     "https://vietnamnet.vn/cong-nghe/san-pham",
     "https://vietnamnet.vn/cong-nghe/vien-thong",
     "https://vietnamnet.vn/cong-nghe/bao-mat", ],
    "Bất động sản":
    ["https://vietnamnet.vn/bat-dong-san",
     "https://vietnamnet.vn/bat-dong-san/du-an",
     "https://vietnamnet.vn/bat-dong-san/noi-that",
     "https://vietnamnet.vn/bat-dong-san/kinh-nghiem-tu-van",
     "https://vietnamnet.vn/bat-dong-san/thi-truong",
     "https://vietnamnet.vn/bat-dong-san/nha-dep",
     "https://vietnamnet.vn/bat-dong-san/kim-oanh-group", ],
    "Bạn đọc":
    ["https://vietnamnet.vn/ban-doc",
     "https://vietnamnet.vn/ban-doc/hoi-am",
     "https://vietnamnet.vn/ban-doc/chia-se",
     "https://vietnamnet.vn/ban-doc/tho",
     "https://vietnamnet.vn/ban-doc/ngay-mai-tuoi-sang", ]
}


In [3]:
for key, value in vietnamnet.items():
    url_array = []
    for url in value:
        for i in range(55,65):
            url_array.append(url + "-page" + str(i))
    vietnamnet[key] += url_array


In [4]:
vietnamnetLinks = {}
for key in vietnamnet:
    vietnamnetLinks[key] = []
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
for key, value in vietnamnet.items():
    for url in value:
        try:
            response = requests.get(url, headers=headers)
            tree = lxml.html.fromstring(response.content)
            title_elem = tree.cssselect('.vnn-title a')
            for elem in title_elem:
                vietnamnetLinks[key].append("https://vietnamnet.vn"+elem.get('href'))
        except:
            pass

for key in vietnamnetLinks:
    print(key, len(vietnamnetLinks[key]))

Đời sống 2382
Thời sự 1666
Quốc tế 2126
Kinh doanh 2660
Giải trí 3072
Thể thao 1682
Sức khỏe 1508
Pháp luật 1101
Oto - xe máy 1063
Công nghệ 2422
Bất động sản 2447
Bạn đọc 1218


In [5]:
vietnamnetContent = {}
for key in vietnamnet:
    vietnamnetContent[key] = []
for key, value in vietnamnetLinks.items():
    for url in value:
        if(url.startswith('https://vietnamnet.vn/')):
            try:
                id = url.split('-')[-1].replace('.html', '')
                downloaded = trafilatura.fetch_url(url)
                tree = lxml.html.fromstring(str(downloaded))
                date_elem = len(tree.cssselect("div.breadcrumb-box__time p span")
                                ) and tree.cssselect("div.breadcrumb-box__time p span")[0].text or ""
                content = trafilatura.extract(downloaded)
                content = content.replace('\n', ' ')
                item = {"domain": "https://vietnamnet.vn/", "URL": url,
                        "content": content, "date": date_elem, "id": id}
                vietnamnetContent[key].append(item)
            except:
                pass

# for key in vietnamnetContent:
    print(key, len(vietnamnetContent[key]))

Đời sống 1242
Thời sự 1096
Quốc tế 1242
Kinh doanh 1520
Giải trí 1552
Thể thao 352
Sức khỏe 938
Pháp luật 531
Oto - xe máy 303
Công nghệ 1352
Bất động sản 1523
Bạn đọc 647


In [6]:
client = Elasticsearch("http://localhost:9200")

client.info()


/home/anh-nh/.local/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'anhnh-Vostro-3578',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'j_KLsaXnT3u6trvp-MtsaA',
 'version': {'number': '7.17.3',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff',
  'build_date': '2022-04-19T08:11:19.070913226Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [7]:
def gendata():
    for key,value in vietnamnetContent.items():
         for item in value:
            temp_item = {k: item[k] for k in set(list(item.keys())) - set(['id'])}
            temp_item['category'] = key
            client.index(index="journal", id=item['id'], document=temp_item)

gendata()